# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [6]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

12


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [27]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [28]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print(e)    

#### Set Keyspace

In [29]:
# Set KEYSPACE to udacity
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

The first query requires us to return data about song's artist, title and length. In our create table statement we will include these three fields. In addition to the fields we also need to add `sessionId` and `itemInSession` fields because they are used in the restriction of the data. 

To create a composite primary key I have grouped `sessionId` and `itemInSession`. The order of the columns in table creation is important because it impacts clustering restrictions. Since we are restricting by `sessionId` and `itemInSession` those two fields are first and second followed by artist, song and length. 

Last the insert statement has the columns also following the order of table creation. 

In [30]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## select artist, song, length from music_library where sessionId = 338 and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(sessionId int, itemInSession int, artist text, song text, length double, PRIMARY KEY(sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [31]:
## Insert into music_app_history data from event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Build INSERT statement
        query = "INSERT INTO music_app_history(sessionId, itemInSession, artist, song, length) "
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign column element for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [41]:
## SELECT statement for sessionId 338, itemInSession 4 and display artist, song and length
query = """
    select artist, 
        song, 
        length 
    from music_app_history 
    where sessionId = 338 
        and itemInSession = 4
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

The second query requires us to return the name of artist, song and the user's first and last name. In the create table statement I will include these four fields namely artist, song, firstName and lastName. Like the first query there is restriction with userid and sessionid so they too will be added in the create table statement. 

My composite primary key will be userid and sessionid. Since we want to sort by itemInSession I added it as part of the clustering column. Like the first query order of the create table is important so I have userid, sessionId and itemInSession as the first three respectively. I then follow with artist, song, firstName and lastName which are columns to return. 

Last the insert statement has the columns also following the order of table creation.

In [34]:
## Display name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS music_history_display_artist_song_user "
query = query + "(userid int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY((userid, sessionId), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [35]:
## Insert into music_history_display_artist_song_user table data from event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Build INSERT statement
        query = "INSERT INTO music_history_display_artist_song_user(userid, sessionId, itemInSession, artist, song, firstName, lastName) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign column element for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [40]:
## SELECT statement for sessionId 182, userid 10 and display artist, song, firstName and lastName
query = """
    select artist, 
        song, 
        firstName, 
        lastName 
    from music_history_display_artist_song_user 
    where userid = 10 and sessionid = 182
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

The third query requires only the first and last name; however, it will be filtered by song title. First we want to create a composite primary key to create unique rows which I have used song title and userid. Like the first and second query the order of the create table is important so I have song and user id as the first and second columns followed by firstName and lastName. 

For the query we also need to restrict by userId as without it will complain and require `ALLOW FILTERING`. 

Query message if I ran restricting only by song.

`select firstname, lastname from music_history_first_and_last_name where song='All Hands Against His Own'`

```
Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"
```

Unfortunately in the data there is no song by `All Hands Against His Own` so I gave an arbitrary userId = 1. 

Last the insert statement has the columns also following the order of table creation.

In [37]:
## Display user name (first and last) in music_all_hands_against_his_own_first_and_last_name who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS music_history_first_and_last_name "
query = query + "(song text, userid int, firstName text, lastName text, PRIMARY KEY((song, userid)))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [38]:
## Insert into music_history_first_and_last_name table data from event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Build INSERT statement
        query = "INSERT INTO music_history_first_and_last_name(song, userid, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Assign column element for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [39]:
query = """
    select firstname, 
        lastname 
    from music_history_first_and_last_name 
    where song='All Hands Against His Own' 
        and userId = 1
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

### Drop the tables before closing out the sessions

In [24]:
## Drop the table before closing out the sessions
queries_drop_tables = ('drop table music_app_history'
                     , 'drop table music_history_display_artist_song_user'
                     , 'drop table music_history_first_and_last_name')
for query in queries_drop_tables:
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()